# 분류-EllipticEnvelope

## import module & data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import module
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
# classifier / metrics
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.naive_bayes import GaussianNB
#import lightgbm as lgb
#from xgboost import XGBClassifier 
#from sklearn.svm import SVC
# from catboost  import CatBoostClassifier, Pool

In [ ]:
# 추가 import
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
from tqdm import tqdm, tqdm_notebook
from collections import Counter

import torch
from sklearn.covariance import EllipticEnvelope

from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.metrics import average_precision_score, precision_recall_curve, plot_precision_recall_curve

In [ ]:
train=pd.read_csv('train_final.csv')
valid=pd.read_csv('valid_final.csv')

In [ ]:
test=pd.read_csv("test_final.csv")
test.head()

,item_id,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,user_id,gender,credit_score,yearly_income,desired_amount,existing_loan_cnt,existing_loan_amt,age,avg_loan_limit,avg_loan_rate,insert_time,insert_day,age_group,income_type,employment_type,houseown_type,purpose,personal_rehabilitation,insert_month,insert_weekend,is_applied,bank_score,product_score,expected_loan_amt,total_expected_loan,expected_redeem,ID,cluster,insert_days_count,insert_dayofweek,micro_lrate,mortgage_lrate,group_lrate,base_rate,consumerIdx_rate,sentimentIdx_rate
0,7,4,2022-06-20 10:24:38,7,220,10000000.0,11.4,209531.0,0.0,720.0,26000000.0,16000000.0,5.0,67000000.0,45.0,1.800000e+07,11.400000,2022-06-20 10:24:36,20.0,40.0,EARNEDINCOME,정규직,기타가족소유,생활비,3.0,6.0,0.0,NaN,0.007589,0.002573,10000000.0,77000000.0,1.032626e+08,2095314,4.0,112,0,0.000000,0.0,0.0,0.285714,-0.06,-0.039
1,8,4,2022-06-20 10:24:38,7,191,26000000.0,11.4,209531.0,0.0,720.0,26000000.0,16000000.0,5.0,67000000.0,45.0,1.800000e+07,11.400000,2022-06-20 10:24:36,20.0,40.0,EARNEDINCOME,정규직,기타가족소유,생활비,3.0,6.0,0.0,NaN,0.007589,0.005016,16000000.0,83000000.0,1.652201e+08,2095314,4.0,112,0,0.000000,0.0,0.0,0.285714,-0.06,-0.039
2,24,8,2022-06-02 13:10:29,30,85,13000000.0,15.9,675741.0,1.0,590.0,9000000.0,5000000.0,5.0,50000000.0,40.0,7.666667e+06,16.766667,2022-06-02 13:10:28,2.0,40.0,PRIVATEBUSINESS,기타,전월세,사업자금,3.0,6.0,0.0,NaN,0.020175,0.007195,5000000.0,55000000.0,7.068856e+07,6757418,4.0,94,3,-0.928571,0.0,0.0,0.285714,0.00,-0.039
3,25,8,2022-06-02 13:10:29,35,29,5000000.0,14.5,675741.0,1.0,590.0,9000000.0,5000000.0,5.0,50000000.0,40.0,7.666667e+06,16.766667,2022-06-02 13:10:28,2.0,40.0,PRIVATEBUSINESS,기타,전월세,사업자금,3.0,6.0,0.0,NaN,0.062279,0.021094,5000000.0,55000000.0,6.475874e+07,6757418,4.0,94,3,-0.528571,0.0,0.0,0.285714,0.00,-0.039
4,26,8,2022-06-02 13:10:30,55,159,5000000.0,19.9,675741.0,1.0,590.0,9000000.0,5000000.0,5.0,50000000.0,40.0,7.666667e+06,16.766667,2022-06-02 13:10:28,2.0,40.0,PRIVATEBUSINESS,기타,전월세,사업자금,3.0,6.0,0.0,NaN,0.017896,0.001798,5000000.0,55000000.0,8.763221e+07,6757418,4.0,94,3,-2.071429,0.0,0.0,0.285714,0.00,-0.039


In [ ]:
X_train = train.drop('is_applied',axis=1)
y_train = train['is_applied']
X_valid = valid.drop('is_applied',axis=1)
y_valid = valid['is_applied']

In [ ]:
print(X_train.shape)
print(test.shape)
print(X_valid.shape)

(3255482, 42)
(3255482, 43)
(1395207, 42)


# 1) Tree-base models

### i. Preprocessing (완)

In [ ]:
train_id = X_train[['user_id','application_id','product_id']]
valid_id = X_valid[['user_id','application_id','product_id']]
test_id = test[['user_id','application_id','product_id']]

In [ ]:
print(X_train.shape)
print(X_valid.shape)
print(test.shape)

(3255482, 42)
(1395207, 42)
(3255482, 43)


In [ ]:
# robust+minmax - 선형 모델
def preprocessing_nontree(X_train, X_valid, X_test):
  ### 0) 칼럼 drop
  # id 인코딩 시 칼럼수 및 희소행렬 증가, 일부 변수 수치형 score로 대체 
    X_train.drop(['item_id','user_id','application_id','bank_id','product_id','ID'],axis=1, inplace=True)
    X_valid.drop(['item_id','user_id','application_id','bank_id','product_id','ID'],axis=1, inplace=True)
    X_test.drop(['item_id','user_id','application_id','bank_id','product_id','ID'],axis=1, inplace=True)
  # datetime형 변수는 파생변수 - month, day, date 생성한 후 drop
  # 이전 데이터 모델링 시도에서 변수중요도가 낮았으며 칼럼수 증가를 방지하기 위해 insert_day도 삭제
  # insert_month는 금리 변동률로 수치형으로 구별돼 라밸링되었기 때문에 역시 drop
    X_train.drop(['loanapply_insert_time','insert_time','insert_month'],axis=1, inplace=True)
    X_valid.drop(['loanapply_insert_time','insert_time','insert_month'],axis=1, inplace=True)
    X_test.drop(['loanapply_insert_time','insert_time','insert_month'],axis=1, inplace=True)
  # 이전 데이터 모델링 시도에서 age가 age_group보다 변수중요도가 높았으며 칼럼수 증가를 방지하기 위해 age_group도 삭제
    X_train.drop(['age_group'],axis=1, inplace=True)
    X_valid.drop(['age_group'],axis=1, inplace=True)
    X_test.drop(['age_group','is_applied'],axis=1, inplace=True)

  # 공통
  # 1) scaler
    from sklearn.preprocessing import RobustScaler, MinMaxScaler, OneHotEncoder
    rbst = RobustScaler()
    mm = MinMaxScaler()
    ohe = OneHotEncoder(sparse = False) 

  # 2) numeric
  ## log
    log_list = ["loan_limit", "yearly_income", "desired_amount", "existing_loan_cnt", 
              "existing_loan_amt","age", "avg_loan_limit", "avg_loan_rate", "total_expected_loan"]
    for col in log_list :
        X_train[col] = np.log1p(X_train[col])
    for col in log_list :
        X_valid[col] = np.log1p(X_valid[col])
    for col in log_list :
        X_test[col] = np.log1p(X_test[col])
    
  ## robust+minmax
    num_features = ["age", "yearly_income", "desired_amount", "existing_loan_cnt", "existing_loan_amt",'credit_score', # user_spec
                  "avg_loan_limit", "avg_loan_rate", "total_expected_loan","loan_limit",'loan_rate', # loan_result
                  'expected_loan_amt', 'expected_redeem', 'bank_score','product_score', 'total_expected_loan','insert_days_count', # 파생변수
                  'mortgage_lrate','micro_lrate','base_rate','group_lrate','consumerIdx_rate','sentimentIdx_rate']

    X_train[num_features] = rbst.fit_transform(X_train[num_features])
    X_valid[num_features] = rbst.transform(X_valid[num_features])
    X_test[num_features] = rbst.transform(X_test[num_features])

  # 3) categorical
  ## onehot
    cat_features = ['gender','income_type','employment_type','houseown_type','purpose','cluster','insert_weekend']
    cat_cols_t = pd.DataFrame(ohe.fit_transform(X_train[cat_features]))
    cat_cols_t.columns = ohe.get_feature_names(cat_features)
    X_train = X_train.drop(cat_features, axis=1)
    X_train = pd.concat([X_train,cat_cols_t], axis=1)

    cat_cols_v = pd.DataFrame(ohe.transform(X_valid[cat_features]))
    cat_cols_v.columns = ohe.get_feature_names(cat_features)
    X_valid = X_valid.drop(cat_features, axis=1)
    X_valid = pd.concat([X_valid,cat_cols_v], axis=1)

    cat_cols_v = pd.DataFrame(ohe.transform(X_test[cat_features]))
    cat_cols_v.columns = ohe.get_feature_names(cat_features)
    X_test = X_test.drop(cat_features, axis=1)
    X_test = pd.concat([X_test,cat_cols_v], axis=1)

  ## minmax scaler
    colnames_t = X_train.columns.tolist()
    X_train = pd.DataFrame(mm.fit_transform(X_train))
    X_train.columns = (colnames_t)

    colnames_v = X_valid.columns.tolist()
    X_valid = pd.DataFrame(mm.transform(X_valid))
    X_valid.columns = (colnames_v)

    colnames_v = X_test.columns.tolist()
    X_test = pd.DataFrame(mm.transform(X_test))
    X_test.columns = (colnames_v)

    return X_train, X_valid, X_test

In [ ]:
X_train, X_valid, X_test = preprocessing_nontree(X_train, X_valid, test)

## 상관계수 확인(전처리 똑같고 순서만 바뀜 + 상관관계 높은 existing_loan_amt drop)

In [ ]:
train1=pd.concat([X_train,y_train],axis=1)

In [ ]:
corr_abs=train1.corr().abs()
s = corr_abs.unstack()
so = s.sort_values(kind="quicksort")

In [ ]:
so[(so>0.5)&(so<1.0)].drop_duplicates()

expected_loan_amt    avg_loan_limit       0.529673
avg_loan_limit       credit_score         0.530669
avg_loan_rate        loan_rate            0.543023
loan_limit           avg_loan_limit       0.579009
insert_days_count    sentimentIdx_rate    0.633835
loan_limit           expected_redeem      0.686693
                     expected_loan_amt    0.748577
expected_loan_amt    expected_redeem      0.923323
total_expected_loan  existing_loan_amt    0.986504
dtype: float64

In [ ]:
train1.drop(['existing_loan_amt'],axis=1, inplace=True)
X_valid.drop(['existing_loan_amt'],axis=1, inplace=True)
X_test.drop(['existing_loan_amt'],axis=1, inplace=True)

In [ ]:
so[(so>0.5)&(so<1.0)].drop_duplicates()

expected_loan_amt          desired_amount              0.505307
employment_type_기타         income_type_FREELANCER      0.506195
credit_score               avg_loan_limit              0.507514
income_type_EARNEDINCOME2  income_type_EARNEDINCOME    0.517609
cluster_2.0                existing_loan_cnt           0.521852
avg_loan_rate              loan_rate                   0.538143
loan_limit                 avg_loan_limit              0.540553
houseown_type_전월세          houseown_type_자가            0.553053
                           houseown_type_기타가족소유        0.557694
cluster_6.0                cluster_4.0                 0.571243
total_expected_loan        existing_loan_cnt           0.585046
income_type_OTHERINCOME    employment_type_기타          0.595753
insert_days_count          sentimentIdx_rate           0.633835
employment_type_계약직        employment_type_정규직         0.637602
employment_type_정규직        employment_type_기타          0.643114
employment_type_기타         income_type_E

In [ ]:
so['is_applied']

income_type_PRACTITIONER       0.000046
cluster_0.0                    0.000449
cluster_3.0                    0.001270
cluster_1.0                    0.001327
purpose_사업자금                   0.001726
base_rate                      0.002981
houseown_type_배우자              0.004089
sentimentIdx_rate              0.004378
houseown_type_전월세              0.004581
cluster_6.0                    0.005329
cluster_5.0                    0.006340
insert_day                     0.007886
product_score                  0.010119
insert_days_count              0.012426
purpose_자동차구입                  0.016450
purpose_기타                     0.017164
cluster_2.0                    0.017859
personal_rehabilitation        0.018628
gender_1.0                     0.018649
gender_0.0                     0.018649
insert_dayofweek               0.019430
employment_type_계약직            0.019980
bank_score                     0.020562
insert_weekend_0.0             0.022549
insert_weekend_1.0             0.022549


In [ ]:
print(train1.shape)
print(X_test.shape)
print(X_valid.shape)

(3255482, 59)
(3255482, 58)
(1395207, 58)


In [ ]:
valid['is_applied']

0          0.0
1          1.0
2          0.0
3          0.0
4          0.0
          ... 
1395202    0.0
1395203    0.0
1395204    0.0
1395205    0.0
1395206    1.0
Name: is_applied, Length: 1395207, dtype: float64

In [ ]:
train[train['user_id'] == 17][['application_id','is_applied']]

,application_id,is_applied
2988660,1990215,1.0
2988661,1990215,0.0
3192520,2125512,1.0


In [ ]:
valid[valid['user_id']==17][['application_id','is_applied']]

,application_id,is_applied
668919,2125512,0.0


In [ ]:
## 데이터 순서 셔플
#from sklearn.utils import shuffle
#X_train, y_train = shuffle(X_train, y_train)
#_valid, y_valid = shuffle(X_valid, y_valid)

In [ ]:
X_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1395207 entries, 0 to 1395206
Data columns (total 61 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   loan_limit                   1395207 non-null  float64
 1   loan_rate                    1395207 non-null  float64
 2   gender                       1395207 non-null  float64
 3   credit_score                 1395207 non-null  float64
 4   yearly_income                1395207 non-null  float64
 5   desired_amount               1395207 non-null  float64
 6   existing_loan_cnt            1395207 non-null  float64
 7   existing_loan_amt            1395207 non-null  float64
 8   age                          1395207 non-null  float64
 9   avg_loan_limit               1395207 non-null  float64
 10  avg_loan_rate                1395207 non-null  float64
 11  personal_rehabilitation      1395207 non-null  float64
 12  bank_score                   1395207 non-n

## Elliptic

In [ ]:
def get_pred_label(model_pred):
    # 0: 제출 x, 1: 제출 o
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

In [ ]:
val_normal, val_fraud = valid['is_applied'].value_counts()
val_contamination = val_fraud / val_normal
print(f'Validation contamination : [{val_contamination}]')

Validation contamination : [0.05714157988531526]


In [ ]:
train_x_ee=train1[train1['is_applied']==0]
train_x_ee.drop(['is_applied'],axis=1,inplace=True)
print(train_x_ee.shape)

(2776349, 58)


In [ ]:
print(X_valid.shape)
print(y_valid.shape)

(1395207, 58)
(1395207,)


**모델 적용**

In [ ]:
model = EllipticEnvelope(support_fraction = 0.994, contamination = val_contamination, random_state = 42)
model.fit(train_x_ee)

NameError: name 'val_x' is not defined

In [ ]:
val_pred = model.predict(X_valid) # model prediction
val_pred = get_pred_label(val_pred)Z

In [ ]:
val_score = f1_score(y_valid, val_pred, average='macro')
print(f'Validation F1 Score : [{val_score}]')
print(classification_report(y_valid, val_pred))

Validation F1 Score : [0.5387267392407559]
              precision    recall  f1-score   support

         0.0       0.95      0.94      0.95   1319792
         1.0       0.12      0.14      0.13     75415

    accuracy                           0.90   1395207
   macro avg       0.54      0.54      0.54   1395207
weighted avg       0.91      0.90      0.90   1395207



In [ ]:
pred=model.predict(X_test) # model prediction
pred = get_pred_label(pred)

In [ ]:
print(test_id.shape)
print(pred.shape)

(3255482, 3)
(3255482,)


### 스태킹 추출용 함수
* 긁어다가 각 모델에 맞게 사용하세요
* 한가지 파일에 추출하는거라, 꼬이지 않게, 추출할때 말씀 남겨주세요!

In [ ]:
# import data
stacking_valid = pd.read_csv('stacking_valid.csv')
stacking_test = pd.read_csv('stacking_test.csv')

In [ ]:
stacking_valid = pd.DataFrame()
stacking_test = pd.DataFrame()

In [ ]:
y_pred_valid = model.predict(X_valid)
y_pred_test = model.predict(X_test)

stacking_valid['EllipticEnvelope'] = y_pred_valid
stacking_test['EllipticEnvelope'] = y_pred_test

In [ ]:
val_pred = get_pred_label(y_pred_valid)

In [ ]:
val_elliptic=valid_id.copy()
val_elliptic['EllipticEnvelope']= val_pred


In [ ]:
val_elliptic['EllipticEnvelope'].value_counts()

0    1308850
1      86357
Name: EllipticEnvelope, dtype: int64

In [ ]:
val_elliptic.to_csv('val_elliptic.csv',index=False)

In [ ]:
# 파일로 추출
stacking_valid.to_csv('stacking_valid_new.csv', index=False)
stacking_test.to_csv('stacking_test_new.csv', index=False)